In [10]:
#--------cell 1--------#
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import traci  # SUMO's Python API
import sumolib  # SUMO utilities

# Load Traffic Dataset
file_path = r"Banglore_traffic_Dataset.csv"
data = pd.read_csv(file_path)

# Extract intersections and traffic data
intersections = data['Road/Intersection Name'].unique()
num_intersections = len(intersections)
traffic_data = {data['Road/Intersection Name'][i]: data['Traffic Volume'][i] for i in range(len(data))}

# SUMO Network and Simulation Setup
sumo_binary = "sumo-gui"  # Use "sumo" for CLI mode
sumo_config = "bangalore.sumocfg"  # SUMO config file for your network

# Start SUMO simulation
traci.start([sumo_binary, "-c", sumo_config])

# Extract SUMO intersections (traffic light IDs)
sumo_traffic_lights = traci.trafficlight.getIDList()
num_intersections = len(sumo_traffic_lights)

#--------cell 2--------#
# Particle Class (Traffic Signal Configuration)
class Particle:
    def __init__(self, num_intersections):
        self.position = np.random.uniform(10, 60, num_intersections)  # Green time (10s-60s)
        self.velocity = np.zeros(num_intersections)
        self.p_best = self.position.copy()
        self.p_best_score = float('inf')

# Fitness Function (Evaluate Traffic Signal Efficiency)
def fitness_function(position):
    total_wait_time = 0
    for i, tl in enumerate(sumo_traffic_lights):
        traci.trafficlight.setPhaseDuration(tl, position[i])  # Apply new timing
        total_wait_time += traci.edge.getWaitingTime(tl)  # Get waiting time
    return total_wait_time  # Lower values are better (less congestion)

# Swarm Initialization
class Swarm:
    def __init__(self, swarm_size, num_intersections):
        self.particles = [Particle(num_intersections) for _ in range(swarm_size)]
        self.global_best_position = np.random.uniform(10, 60, num_intersections)
        self.global_best_score = float('inf')

#--------cell 3--------#
# PSO Algorithm
swarm_size = 10
max_iterations = 50

# Dynamic inertia weight for better convergence
w_max, w_min = 0.9, 0.4
c1, c2 = 1.2, 1.2

swarm = Swarm(swarm_size, num_intersections)
best_scores = []

for iteration in range(max_iterations):
    w = w_max - (w_max - w_min) * (iteration / max_iterations)  # Adaptive inertia weight

    for particle in swarm.particles:
        # Compute fitness (real-time from SUMO)
        score = fitness_function(particle.position)

        # Update personal & global bests
        if score < particle.p_best_score:
            particle.p_best = particle.position.copy()
            particle.p_best_score = score
        if score < swarm.global_best_score:
            swarm.global_best_position = particle.position.copy()
            swarm.global_best_score = score

    best_scores.append(swarm.global_best_score)  # Store for plotting

    # Plot optimization progress
    plt.figure(figsize=(8, 4))
    plt.plot(range(len(best_scores)), best_scores, marker='o', linestyle='-', color='blue')
    plt.xlabel("Iteration")
    plt.ylabel("Best Fitness Score")
    plt.title(f"Iteration {iteration + 1}: Optimization Progress")
    plt.grid()
    plt.show()

    #--------cell 4--------#
    # Update Velocity & Position
    for particle in swarm.particles:
        r1, r2 = np.random.rand(), np.random.rand()
        particle.velocity = (w * particle.velocity) + (c1 * r1 * (particle.p_best - particle.position)) + (c2 * r2 * (swarm.global_best_position - particle.position))
        particle.position += particle.velocity

        # Ensure values stay within limits with some randomness
        particle.position = np.clip(particle.position, 10, 60)

#--------cell 5--------#
# Apply optimized signal timings in SUMO
optimized_times = swarm.global_best_position
optimized_traffic_signals = {sumo_traffic_lights[i]: optimized_times[i] for i in range(num_intersections)}

print("\nOptimized Traffic Signal Timings:")
for key, value in optimized_traffic_signals.items():
    print(f"{key}: {value:.2f} seconds")
    traci.trafficlight.setPhaseDuration(key, value)  # Apply in SUMO

# Final Convergence Curve
plt.figure(figsize=(10, 5))
plt.plot(range(len(best_scores)), best_scores, marker='o', linestyle='-', color='red')
plt.xlabel("Iteration")
plt.ylabel("Best Fitness Score")
plt.title("Final Optimization Convergence Curve")
plt.grid()
plt.show()

# Plot optimized signal timings
plt.figure(figsize=(10, 5))
plt.bar(optimized_traffic_signals.keys(), optimized_traffic_signals.values(), color='skyblue')
plt.xlabel("Intersections")
plt.ylabel("Optimized Green Time (seconds)")
plt.title("Swarm-Based Traffic Signal Optimization with SUMO")
plt.xticks(rotation=45)
plt.show()

# Stop SUMO simulation
traci.close()


ModuleNotFoundError: No module named 'traci'